In [1]:
import pandas as pd
from pathlib import Path
import ast
import os
from datetime import date
from nltk.corpus import stopwords
import nltk
import calendar

In [2]:
stopW = stopwords.words('english')

def contains_european(x):
    y = x.lower()
    words = y.split()
    flag = 'e.u' in words or 'eu' in words or 'europeanunion' in words
    words = [word for word in words if word not in stopW]
    bigrams = list(nltk.bigrams(words))
    bigrams = [bigram[0] + '.' + bigram[1] for bigram in bigrams]
    
    flag = flag or 'european.union' in bigrams or 'europe.union' in bigrams
    
    return 'EU' if flag else 'non_EU'

In [61]:
year = 2016

In [68]:
news_df = pd.read_csv('../data/news_predictions/news_{}_predictions.csv'.format(year))
news_df['top1_topic'] = news_df.apply(lambda x: "_".join(x.top1_topic.replace(',', '').split(" ")), axis=1)
# news_df = news_df.loc[news_df.top1_acc >= 40]
# news_df = news_df.loc[news_df.top1_topic != 'Others']
news_df['Date'] = news_df.apply(lambda x: date(x['year'], x['month'], x['day']).strftime('%B %d, %Y'), axis=1)
news_df = news_df.drop(['Unnamed: 0', 'day', 'month', 'year', 'parliament', 'top1_acc', 'top2_topic', 'top2_acc', 'top3_topic', 'top3_acc'], axis=1)
news_df.columns = ['source_id', 'Source', 'Program Name', 'Transcript', 'topic', 'Date']
news_df = news_df.dropna(subset=['Transcript'])
news_df['eu'] = news_df['Transcript'].apply(lambda x: contains_european(x))

# Phrase Split

In [8]:
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
month_short_name = {
    'January': 'jan',
    'February': 'feb',
    'March': 'mar',
    'April': 'apr',
    'May': 'may',
    'June': 'jun',
    'July': 'jul',
    'August': 'aug',
    'September': 'sep',
    'October': 'oct',
    'November': 'nov',
    'December': 'dec'
}
short_names = month_short_name.values()

topics = ['EU', 'non_EU']
ids = [389195, 138620, 397135, 8200]
years = [2014, 2015, 2016, 2017, 2018]


for news_id in ids:
    for year in years:
        print(news_id, year)
        for month in short_names:
            df_path = '/home/ubuntu/Nexis/{}/{}/ {}-{} {}.csv'.format(news_id, year, month, year, news_id)
            news_df = pd.read_csv(df_path)
            news_df = news_df.drop(['Unnamed: 0'], axis=1)
            news_df.columns = ['Source', 'Date', 'Program Name', 'Transcript']
            news_df = news_df.dropna(subset=['Transcript'])
            news_df['eu'] = news_df['Transcript'].apply(lambda x: contains_european(x))
            for topic in topics:
                path = './news_topics/{}_{}/{}'.format(news_id, topic, year)
                Path(path).mkdir(parents=True, exist_ok=True)
                news_for_topic = news_df.loc[news_df.eu == topic]
                news_for_topic.to_csv(os.path.join(path, '{}-{} {}_{}.csv'.format(month, year, news_id, topic)))

389195 2014
389195 2015
389195 2016
389195 2017
389195 2018
138620 2014
138620 2015
138620 2016
138620 2017
138620 2018
397135 2014
397135 2015
397135 2016
397135 2017
397135 2018
8200 2014
8200 2015
8200 2016
8200 2017
8200 2018


In [58]:
rows = []

for topic in topics:
    for news_id in news_ids:
        row = [id_to_name_map[news_id]+ ' ' +topic, str(news_id)+'_'+topic, 'Nexis', '', '']
        rows.append(row)
        print(row)
        
df = pd.DataFrame(rows, columns=['Source Name', 'SourceId', 'Database', 'Start', 'End'])
df

['Belfast Telegraph EU', '400553_EU', 'Nexis', '', '']
['The Scotsman EU', '377101_EU', 'Nexis', '', '']
['Daily Record EU', '418973_EU', 'Nexis', '', '']
['Wales on Sunday EU', '244365_EU', 'Nexis', '', '']
['Independent EU', '8200_EU', 'Nexis', '', '']
['Wales Online EU', '412338_EU', 'Nexis', '', '']
['Mail on Sunday EU', '138794_EU', 'Nexis', '', '']
['Sunday Express EU', '232241_EU', 'Nexis', '', '']
['Sunday Telegraph EU', '334988_EU', 'Nexis', '', '']
['Sunday Sun EU', '331369_EU', 'Nexis', '', '']
['Guardian EU', '138620_EU', 'Nexis', '', '']
['Mirror EU', '419001_EU', 'Nexis', '', '']
['Guardian Weekly EU', '8010_EU', 'Nexis', '', '']
['The Herald EU', '142728_EU', 'Nexis', '', '']
['Express Online EU', '408506_EU', 'Nexis', '', '']
['The Observer EU', '143296_EU', 'Nexis', '', '']
['Daily Star Sunday EU', '363952_EU', 'Nexis', '', '']
['The People EU', '145251_EU', 'Nexis', '', '']
['The Express EU', '232240_EU', 'Nexis', '', '']
['Daily Record and Sunday Mail EU', '145253_EU

,Source Name,SourceId,Database,Start,End
0,Belfast Telegraph EU,400553_EU,Nexis,,
1,The Scotsman EU,377101_EU,Nexis,,
2,Daily Record EU,418973_EU,Nexis,,
3,Wales on Sunday EU,244365_EU,Nexis,,
4,Independent EU,8200_EU,Nexis,,
5,Wales Online EU,412338_EU,Nexis,,
6,Mail on Sunday EU,138794_EU,Nexis,,
7,Sunday Express EU,232241_EU,Nexis,,
8,Sunday Telegraph EU,334988_EU,Nexis,,
9,Sunday Sun EU,331369_EU,Nexis,,


In [126]:
df.to_csv('./sources2.csv')